# Proyecto Grupal - Etapa 04: Limpieza de Datos
Autores:
Camila de la Paz
Daniel E. Ramírez
Franco Pes
Xavier Vidman

El presente archivo compila la tercera etapa del primer proyecto grupal de la carrera de Data Science de Henry, un análisis exploratorio sobre los datos proporcionados. Esta etapa se divide, a su vez, en los pasos que se detallan a continuación:
1. Importación de liberías a utilizar
2. Carga de datos
3. Limpieza y Transformación

### Paso 1: Importación de librerías

In [32]:
import datetime
import numpy as np
import pandas as pd

### Paso 2: Carga de datos


In [33]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2018-01.parquet')

### Paso 3: Limpieza y Transformación

#### a) Se eliminaron columnas innecesarias

In [34]:
df.drop(columns=['congestion_surcharge','airport_fee'], inplace=True)

#### b) Se eliminaron registros duplicados

In [35]:
df.drop_duplicates(inplace=True)

#### c) Se renombraron las columnas

In [36]:
df.rename(columns =
                    {'VendorID':'id_vendor',
                    'RatecodeID':'id_ratecode',
                    'PULocationID':'id_pu_zone',
                    'DOLocationID':'id_do_zone'}, inplace = True)

#### d) Se creó una nueva columna, fare_per_mile, para estudiar la relación entre fare_amount y trip_distance

In [37]:
df['trip_distance_aux'] = df['trip_distance']
df['trip_distance_aux'].replace(0, 1, inplace=True)
df['trip_distance_aux'].fillna(1, inplace=True)

df['fare_per_mile'] = df.fare_amount / df.trip_distance_aux

df.loc[df['trip_distance'] == 0, 'fare_per_mile'] = 0

df.drop(columns=['trip_distance_aux'], inplace=True)

#### e) Se creó una nueva columna, trip_time, para identificar el tiempo de viaje en segundos

Primero se calculó la diferencia de tiempo

In [38]:
df['trip_time'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime

Luego se convirtió a segundos

In [39]:
df.trip_time = df.trip_time.dt.total_seconds()

#### f) Se creó una nueva columna, fare_per_minute, para identificar la relación entre el fare_amount y el trip_time

In [40]:
df['trip_time_aux'] = df['trip_time']
df['trip_time_aux'].replace(0, 1, inplace=True)
df['trip_time_aux'].fillna(1, inplace=True)

df['fare_per_minute'] = df.fare_amount / (df.trip_time_aux / 60)

df.loc[df['trip_time'] == 0, 'fare_per_minute'] = 0

df.drop(columns=['trip_time_aux'], inplace=True)

#### g) Se comenzó a trabajar con el dataframe de zonas y el dataframe de boroughs

Lo primero es cargar el dataframe con las zonas de taxis

In [41]:
df_zone = pd.read_csv('https://raw.githubusercontent.com/soyHenry/DS-Proyecto_Grupal_TaxisNYC/main/taxi%2B_zone_lookup.csv')

Renombramos columnas

In [42]:
df_zone.rename(columns =
                    {'LocationID':'id_zone',
                    'Borough':'borough',
                    'Zone':'zone'}, inplace = True)

Creamos dataframe de boroughs

In [43]:
borough_data = {'id_borough': [1, 2, 3, 4, 5, 6, 7], 'borough':['Brooklyn', 'Bronx', 'Manhattan', 'Staten Island', 'Queens', 'EWR', 'Unknown']}
df_borough = pd.DataFrame(borough_data)

Se creó un diccionario de zonas con su respectivo borough para luego mapear

In [44]:
dic_zone_borough = {df_zone.id_zone[i] : df_zone.borough[i] for i in range (0,len(df_zone))}

Se creó un diccionario del id_borough con su respectivo borough para luego mapear

In [45]:
dic_id_borough = {df_borough.borough[i] : df_borough.id_borough[i] for i in range (0, len(df_borough))}

Se crean dos nuevas columnas: pu_borough: "pull up borough" y do_borough: "drop off borough"

In [46]:
df['pu_borough'] = df.id_pu_zone.map(dic_zone_borough)
df['do_borough'] = df.id_do_zone.map(dic_zone_borough)

Se crea un id_borough en el dataframe de taxis para luego realizar la conexión

In [47]:
df['id_borough'] = df.pu_borough.map(dic_id_borough)

#### h) Se creó una nueva columna, id_time_borough, para posteriormente relacionar con la tabla de datos climáticos

In [48]:
df['id_time_borough'] = df.tpep_pickup_datetime.dt.strftime('%Y%m%d%H') + df.id_borough.astype(str)

#### i) Se marcaron registros que poseen outliers

Creamos la columna para identificar el outliers

In [49]:
df['outlier'] = 1

Outliers trip_distance

In [50]:
# Calculamos rango intercuartílico, mínimo, y máximo
IQR = df.trip_distance.quantile(.75) - df.trip_distance.quantile(.25)
min = df.trip_distance.quantile(.25) - (1.5 * IQR)
max = df.trip_distance.quantile(.75) + (1.5 * IQR)

# Indentificamos outliers
df.loc[df.trip_distance < min, "outlier"] = 0
df.loc[df.trip_distance > max, "outlier"] = 0

Outliers fare_amount

In [51]:
# Calculamos rango intercuartílico, mínimo, y máximo
IQR = df.fare_amount.quantile(.75) - df.fare_amount.quantile(.25)
min = df.fare_amount.quantile(.25) - (1.5 * IQR)
max = df.fare_amount.quantile(.75) + (1.5 * IQR)

# Indentificamos outliers
df.loc[df.fare_amount < min, "outlier"] = 0
df.loc[df.fare_amount > max, "outlier"] = 0

Outliers trip_time

In [52]:
# Calculamos rango intercuartílico, mínimo, y máximo
IQR = df.trip_time.quantile(.75) - df.trip_time.quantile(.25)
min = df.trip_time.quantile(.25) - (1.5 * IQR)
max = df.trip_time.quantile(.75) + (1.5 * IQR)

# Indentificamos outliers
df.loc[df.trip_time < min, "outlier"] = 0
df.loc[df.trip_time > max, "outlier"] = 0

In [53]:
df['id_trip'] = df.index.values
df['id_trip'] = df['id_trip'] + 1

In [54]:
cols = list(df.columns)
df = df[cols[25:26] + cols[0:25]]

In [ ]:
df['datetime_snow'] = df.tpep_pickup_datetime.dt.strftime('%Y%m%d')

In [59]:
df.to_csv('taxi.csv', index=False)
df_zone.to_csv('zone.csv', index=False)
df_borough.to_csv('borough.csv', index=False)